In [1]:
# softmax_multi_classification
# multi-nomial Classification (다중 분류) : Y값의 범주가 3개 이상인 분류
# 활성화 함수(Activation function) 으로 softmax()가 사용 된다

import tensorflow as tf
import numpy as np
tf.random.set_seed(5)

In [2]:
# 데이터 가져오기
xy = np.loadtxt('data-04-zoo.csv',delimiter=',',dtype=np.float32)

# 학습 데이터 : 70%, 70개
x_train = xy[:70,:-1]
y_data = xy[:70,[-1]]
y_train = y_data.astype('int32')  # one-hot 인코딩을 위해 반드시 int형으로 사용
print(x_train.shape)  # (70,16)
print(y_train.shape)  #(70,1)

# 검증 데이터 : 30%, 31개
x_test = xy[70:,:-1]
y_data2 = xy[70:,[-1]]
y_test = y_data2.astype('int32')   # one-hot 인코딩을 위해 반드시 int형으로 사용
print(x_test.shape)  # (31,16)
print(y_test.shape)  #(31,1)
print(x_train.dtype,x_test.dtype)
print(y_train.dtype,y_test.dtype)

(70, 16)
(70, 1)
(31, 16)
(31, 1)
float32 float32
int32 int32


In [3]:
# one-hot 인코딩
nb_classes = 7  # 분류 class의 갯수(0~6)
Y_one_hot = tf.one_hot(y_train,nb_classes)  # [None,1,7]
print(Y_one_hot.shape)                      #[70,1,7], Rank=3 (3차원)
Y_one_hot = tf.reshape(Y_one_hot,[-1,nb_classes])
print(Y_one_hot.shape)                      # [70,7], Rank=2 (2차원)

(70, 1, 7)
(70, 7)


In [4]:
# 변수 초기화 : weight, bias
# (70,16) *(16,7) = (70,7)
nb_classes = 7  # 분류 갯수

W = tf.Variable(tf.random.normal([16,nb_classes]), name ='weight')
b = tf.Variable(tf.random.normal([nb_classes]), name = 'bias')

In [5]:
# hypothesis 예측 함수 : H(X) = softmax(W*X + b)
def logits(X):
    return tf.matmul(X,W) + b

def hypothesis(X):
    return   tf.nn.softmax(logits(X))
# 위 함수 반환값을 아래 함수에서 사용

In [9]:
# 비용 함수 구현 방법 2 : tf.nn.softmax_cross_entropy_with_logits() 함수 사용
def cost_func():
    cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits(x_train),
                                                     labels=Y_one_hot)
    cost = tf.reduce_mean(cost_i)
    return cost

In [10]:
# 경사 하강법
# learning_rate(학습율) 을 0.01로 설정하여 optimizer 객체를 생성
optimizer = tf.keras.optimizers.Adam(lr=0.01)

In [11]:
# 학습 시작
print('***** Start Learning!!')
for step in range(5001):
    # cost를 minimize한다
    optimizer.minimize(cost_func,var_list=[W,b])
    
    if step % 1000 == 0:
        print('%04d'%step,'cost: [',cost_func().numpy(),']')
        
print('***** Learning Finished')

***** Start Learning!!
0000 cost: [ 3.364605 ]
1000 cost: [ 0.0032419092 ]
2000 cost: [ 0.0009893123 ]
3000 cost: [ 0.00044815178 ]
4000 cost: [ 0.00023481257 ]
5000 cost: [ 0.00013186333 ]
***** Learning Finished


In [13]:
# 회귀 계수 : weight과 bias출력
print('Weight :',W.numpy())
print('Bias:',b.numpy())

Weight : [[ 50.305054  38.481403  49.102955  48.55165   47.386562  45.24111
   47.244205]
 [ 44.422176  55.902668  46.48745   48.136486  46.402138  45.45441
   44.485523]
 [ 46.65957   47.61995   49.68914   52.743843  50.979492  50.671906
   52.682156]
 [-49.219154 -49.281418 -54.504124 -54.825558 -54.915813 -50.94095
  -57.69752 ]
 [ 48.408752  49.857864  45.55915   48.404934  45.902523  51.01263
   44.26856 ]
 [ 48.095367  48.9789    45.45107   53.3811    51.20776   46.188877
   52.60581 ]
 [ 50.219772  49.36512   49.16879   47.69735   50.69365   48.032433
   52.679306]
 [ 51.410393  47.37609   49.91512   50.776722  52.77001   48.867798
   46.30844 ]
 [ 51.618153  49.975452  52.61507   49.982723  52.730465  48.777664
   46.111492]
 [ 50.96331   50.546646  52.38328   45.86567   48.776855  48.84697
   41.655403]
 [ 48.14555   48.796326  51.99006   48.442303  49.943752  49.29761
   46.938416]
 [ 49.480103  49.118008  45.47159   52.46855   44.720966  48.81852
   47.04557 ]
 [ 49.992184  

In [12]:
# 정확도 평가 : accuracy computation
# 학습과 검증데이터를 별도로 사용한 경우 (70%:30%)

# y_test 값의 one-hot 인코딩
Y_one_hot = tf.one_hot(y_test,nb_classes)  # [None,1,7]
print(Y_one_hot.shape)                     # [31,1,7], Rank=3 (3차원)
Y_one_hot = tf.reshape(Y_one_hot,[-1,nb_classes])
print(Y_one_hot.shape)                     # [31,7], Rank=2 (2차원)

def predict(X):
    return tf.argmax(hypothesis(X),1) 

correct_predict = tf.equal(predict(x_test),tf.argmax(Y_one_hot,1))

accuracy = tf.reduce_mean(tf.cast(correct_predict, dtype = tf.float32))

print("Accuracy:",accuracy.numpy()) # Accuracy: 0.8064516

print('***** Predict')
pred = predict(x_test).numpy()
for p,y in zip(pred, y_test.flatten()):
    print("[{}] Prediction: {} / Real Y: {}".format(p == int(y), p, int(y)))

(31, 1, 7)
(31, 7)
Accuracy: 0.83870965
***** Predict
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 1 / Real Y: 1
[False] Prediction: 5 / Real Y: 6
[True] Prediction: 3 / Real Y: 3
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 2 / Real Y: 2
[True] Prediction: 6 / Real Y: 6
[True] Prediction: 1 / Real Y: 1
[True] Prediction: 1 / Real Y: 1
[True] Prediction: 2 / Real Y: 2
[False] Prediction: 2 / Real Y: 6
[True] Prediction: 3 / Real Y: 3
[True] Prediction: 1 / Real Y: 1
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 6 / Real Y: 6
[True] Prediction: 3 / Real Y: 3
[True] Prediction: 1 / Real Y: 1
[True] Prediction: 5 / Real Y: 5
[True] Prediction: 4 / Real Y: 4
[False] Prediction: 5 / Real Y: 2
[False] Prediction: 4 / Real Y: 2
[True] Prediction: 3 / Real Y: 3
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 1 / Real Y: 1
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 5 / Real Y: 5
[True] Prediction: